In [2]:
import tensorflow as tf
import numpy as np
#import matplotlib.pyplot as plt
import cv2
#import pprint
tf.reset_default_graph()  
'''
print(tf.__version__)
print(np.__version__)
print(cv2.__version__)
1.13.1
1.16.2
4.1.0
'''
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)

class cell():
    sess = tf.Session()
    X = tf.placeholder(tf.float32, [None, 784])
    X_img = tf.reshape(X, [-1,28,28,1]) # (black/white)
    Y = tf.placeholder(tf.float32, [None, 10])


    learning_rate=0.001
    training_epochs = 15
    batch_size = 100
    total_batch = int(mnist.train.num_examples / batch_size)

    keep_prob = tf.placeholder(tf.float32)

    W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
    L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding="SAME")
    L1 = tf.nn.relu(L1) 
    L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1],
                           strides=[1,2,2,1], padding="SAME")
    L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

    #L2 ImgIn shape=(?,14,14,32)
    W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01))
    #Conv -> (?,14,14,64), Pool -> (?,7,7,64)
    L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding="SAME")
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
    L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

    #L3 ImgIn shape=(?,7,7,64)
    W3 = tf.Variable(tf.random_normal([3,3,64,128], stddev=0.01))
    #Conv=(?,7,7,128), Pool=(?,4,4,128),
    #Reshape=(?,4*4*128) <- Flatten them for FC input
    L3 = tf.nn.conv2d(L2, W3, strides=[1,1,1,1], padding="SAME")
    L3 = tf.nn.relu(L3)
    L3 = tf.nn.max_pool(L3, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
    L3 = tf.nn.dropout(L3,keep_prob=keep_prob)

    L3 = tf.reshape(L3, [-1, 4*4*128]) #Flatten them for FC input

    #FC 4*4*128 inputs -> 625 outputs
    W4 = tf.get_variable("W4",shape=[4*4*128,625],
                            initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([625]))
    L4 = tf.nn.relu(tf.matmul(L3,W4) + b4)
    L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

    #L5 Final FC 625 inputs -> 10 outputs
    W5 = tf.get_variable("W5",shape=[625,10], initializer=tf.contrib.layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal([10]))
    hypothesis = tf.matmul(L4,W5) + b5

    #defice cost/loss & optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    sess.run(tf.global_variables_initializer())

    print("Learning started.")
    #print("I should use tensorflow GPU...")
    for epoch in range(training_epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feed_dict = {X:batch_xs, Y:batch_ys, keep_prob:0.7}
            c, _, =sess.run([cost, optimizer], feed_dict = feed_dict)
            avg_cost += c / total_batch
        print("Epoch:","%04d"%(epoch+1), "cost =","{:.9f}".format(avg_cost))

    print("Learning Finished!")

    #Test model and check accuracy
    correct_prediction = tf.equal(tf.argmax(hypothesis,1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", sess.run(accuracy, feed_dict = {X:mnist.test.images, Y:mnist.test.labels, keep_prob:1}))

    def predict(self,xdata):
        return self.sess.run(tf.argmax(self.hypothesis,1),feed_dict={self.X:xdata, self.keep_prob:1})
# training result
'''
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started.
Epoch: 0001 cost = 0.431525774
Epoch: 0002 cost = 0.105765128
Epoch: 0003 cost = 0.076961261
Epoch: 0004 cost = 0.061340322
Epoch: 0005 cost = 0.053813879
Epoch: 0006 cost = 0.046879263
Epoch: 0007 cost = 0.041968299
Epoch: 0008 cost = 0.041718365
Epoch: 0009 cost = 0.037297408
Epoch: 0010 cost = 0.031468368
Epoch: 0011 cost = 0.031107485
Epoch: 0012 cost = 0.031315916
Epoch: 0013 cost = 0.027793923
Epoch: 0014 cost = 0.029483450
Epoch: 0015 cost = 0.025088975
Learning Finished!
Accuracy: 0.9936
'''
    
    
    
# cell class for predicting the number image data
net = cell()

def preprocessing1(img):
    # This preprocessing is to take the data of the center of the image 
    # img data I used: img.shape = (480, 640, 3) 
    img = img[190:290,270:370]
    # color -> gray ((100, 100, 3) -> (100,100,1))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Put some blur to make similar data with MNIST data
    img = cv2.GaussianBlur(img, (3, 3), 0)
    # Change the shape of the data to MNIST data shape
    img = cv2.resize(img, (28, 28))
    
    #debugging using matplotlib.pyplot (you can see what kind of data your taking)
    #plt.imshow(img, cmap="Greys", interpolation="nearest")
    #plt.show()
    
    # Use threshold to clearly distinguish the black and white
    res, img = cv2.threshold(img, 110 , 255, cv2.THRESH_BINARY)
    img = 255 - img
    img = img.astype(np.float32)
    # scaling the data
    img /= 255
    # reshape the data to use it as an input of MNIST data model
    img = np.array(img).reshape(1,784)
    return img

def preprocessing2(img):
    # color -> gray ((x,x,3) -> (x,x,1))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.GaussianBlur(img, (3, 3), 0)
    img = cv2.resize(img, (28, 28))
    #debugging using matplotlib.pyplot (you can see what kind of data your taking)
    #plt.imshow(img, cmap="Greys", interpolation="nearest")
    #plt.show()
    
    # Use threshold to clearly distinguish the black and white
    res, img = cv2.threshold(img, 110 , 255, cv2.THRESH_BINARY)
    img = 255 - img
    img = img.astype(np.float32)
    # scaling the data
    img /= 255
    # reshape the data to use it as an input of MNIST data model
    img = np.array(img).reshape(1,784)
    return img

# capture the data from camera
capture = cv2.VideoCapture(0)
# initialize the Window
#cv2.namedWindow("debug_1") # This window is for debugging the extracted data using contour method (showing the contour of whole display)
cv2.namedWindow("predicting number") # This window will automatically extract the data of number written in the paper and predict it
#cv2.namedWindow("predicting number of center") # This window will predict the data of the center

while True:
    # read the data from capture
    ret, frame = capture.read();
    
    # changing to gray scale
    img_gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    
    # Threshold Filtering
    retval, dst = cv2.threshold(img_gray, 110, 255, cv2.THRESH_BINARY_INV )
    # inversion of black and white
    dst = cv2.bitwise_not(dst)
    retval, dst = cv2.threshold(dst, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    
    #extract the contour from the filtered data
    contours, hierarchy = cv2.findContours(dst, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # code for debugging using contour
    #ret, dst = capture.read();
    #dst = cv2.drawContours(dst, contours, -1, (0, 0, 255, 255), 2, cv2.LINE_AA)
    #cv2.imshow('debug_1', dst)
    
    # read the original data 
    ret, dst = capture.read();

    for contour in contours:
        area = cv2.contourArea(contour)
        # only using contours that have area over than 400 and less than 20000 
        if area<400 and 20000>area:
            continue
        
        x,y,w,h = cv2.boundingRect(contour) 
        a = max(w,h)
        # img.shape = (480, 640, 3) 
        if (a<10 or x+w/2-a*1.3/2<1 or x+w/2+a*1.3/2>479 or y+h/2-a*1.3/2 <1 or y+h/2+a*1.3/2 >639):
            continue
        # extract range is decided to make the number data to be similar with MNIST data(train data)
        img = dst[int(y+h/2-a*1.3/2):int(y+h/2+a*1.3/2),int(x+w/2-a*1.3/2):int(x+w/2+a*1.3/2),:]
        img = preprocessing2(img) # img shape => (28,28)
        # drawing rectangle around the data that is predicted
        dst = cv2.rectangle(dst,(int(x+w/2-a*1.3/2),int(y+h/2-a*1.3/2)),(int(x+w/2+a*1.3/2),int(y+h/2+a*1.3/2)),(0,255,0),2)
        # predict the data
        pred = net.predict(img)[0]
        text = "number{}".format(str(pred))
        font = cv2.FONT_HERSHEY_PLAIN
        font_size = 1
        # put the text including the prediction information
        cv2.putText(dst,text,(int(x+w/2-a*1.3/2),int(y+h/2+a*1.3/2 +16)),font, font_size,(255,255,0)) 
        
        c = cv2.waitKey(1)
        
    cv2.imshow('predicting number', dst)
    # get the center image (100,100,3), this is for easy checking 
    #cv2.rectangle(frame,(270,190),(370,290),(0,0,255),3)
    #img = preprocessing1(frame)
    #pred = net.predict(img)[0]
    #text = "number{}".format(str(pred))
    #font = cv2.FONT_HERSHEY_PLAIN
    #font_size = 1
    #cv2.putText(frame,text,(370,290),font, font_size,(255,255,0)) 
            
    #cv2.imshow("predicting number of center", frame)
    c = cv2.waitKey(1)

    if c == 27: # Esc key
        print("Section ended")
        break

capture.release()
cv2.destroyAllWindows()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started.
Epoch: 0001 cost = 0.431525774
Epoch: 0002 cost = 0.105765128
Epoch: 0003 cost = 0.076961261
Epoch: 0004 cost = 0.061340322
Epoch: 0005 cost = 0.053813879
Epoch: 0006 cost = 0.046879263
Epoch: 0007 cost = 0.041968299
Epoch: 0008 cost = 0.041718365
Epoch: 0009 cost = 0.037297408
Epoch: 0010 cost = 0.031468368
Epoch: 0011 cost = 0.031107485
Epoch: 0012 cost = 0.031315916
Epoch: 0013 cost = 0.027793923
Epoch: 0014 cost = 0.029483450
Epoch: 0015 cost = 0.025088975
Learning Finished!
Accuracy: 0.9936
Section ended
